In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from keras._tf_keras.keras.models import Sequential
from keras._tf_keras.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras._tf_keras.keras.utils import to_categorical

2025-01-18 18:03:24.908145: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-18 18:03:24.909021: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-18 18:03:24.912516: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-18 18:03:24.921936: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737241404.939358  116849 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737241404.94

In [2]:
IMAGE_SIZE = (320, 320) 
BATCH_SIZE = 32
EPOCHS = 15
TRAIN_DIR = "train" 
VALIDATION_SPLIT = 0.3 

In [3]:
try:
    categories = sorted(os.listdir(TRAIN_DIR))  #Listed Categories
    if not categories:
        raise ValueError("Training directory is empty.")
except FileNotFoundError:
    raise FileNotFoundError(f"Training directory '{TRAIN_DIR}' does not exist.")

In [4]:
# Function to load in OpenCV instead of Pillow
def load_data_with_cv2(directory, categories, image_size):
    images = []
    labels = []
    for idx, category in enumerate(categories):
        category_path = os.path.join(directory, category)
        if not os.path.isdir(category_path):
            print(f"Warning: '{category_path}' is not a directory. Skipping.")
            continue
        for file in os.listdir(category_path):
            img_path = os.path.join(category_path, file)
            try:
                img = cv2.imread(img_path)  # Load image
                if img is None:
                    print(f"Warning: Could not load image '{img_path}'. Skipping.")
                    continue
                img = cv2.resize(img, image_size)  # Resize to Size set in kernal 2
                img = img / 255.0  # pixel values to [0, 1]
                images.append(img)
                labels.append(idx)
            except Exception as e:
                print(f"Error loading image '{img_path}': {e}")
    if not images:
        raise ValueError("No valid images found in the dataset.")
    return np.array(images), np.array(labels)

In [ ]:
# Load training
train_images, train_labels = load_data_with_cv2(TRAIN_DIR, categories, IMAGE_SIZE)

# Shuffle and split the data manually using NumPy
indices = np.arange(len(train_images))
np.random.seed(42)  # Ensure reproducibility
np.random.shuffle(indices)

# Compute the split index
split_idx = int(len(train_images) * (1 - VALIDATION_SPLIT))

# Split the data
train_indices, val_indices = indices[:split_idx], indices[split_idx:]
train_images, val_images = train_images[train_indices], train_images[val_indices]
train_labels, val_labels = train_labels[train_indices], train_labels[val_indices]

# Convert labels to one-hot encoding
train_labels = to_categorical(train_labels, num_classes=len(categories))
val_labels = to_categorical(val_labels, num_classes=len(categories))

# Create TensorFlow datasets for training and validation
train_data = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_data = train_data.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_data = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_data = val_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

2025-01-18 18:03:29.343914: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(categories), activation='softmax')  # One output per category
])

/home/odruzgal/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 318, 318, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 159, 159, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 159, 159, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 157, 157, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 78, 78, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 78, 78, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 76, 76, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 38, 38, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 184832)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    23,658,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,752,775 (90.61 MB)

 Trainable params: 23,752,775 (90.61 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    verbose=1
)

Epoch 1/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.2087 - loss: 7.4095 - val_accuracy: 0.1386 - val_loss: 1.9456
Epoch 2/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.1728 - loss: 1.9485 - val_accuracy: 0.1881 - val_loss: 1.9455
Epoch 3/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.1859 - loss: 1.9453 - val_accuracy: 0.1980 - val_loss: 1.9438
Epoch 4/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.1634 - loss: 1.9398 - val_accuracy: 0.1980 - val_loss: 1.9440
Epoch 5/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.1761 - loss: 1.9428 - val_accuracy: 0.1980 - val_loss: 1.9321
Epoch 6/25
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2084 - loss: 1.9125

2025-01-18 18:05:30.947268: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 78644992 bytes after encountering the first element of size 78644992 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.2069 - loss: 1.9133 - val_accuracy: 0.1980 - val_loss: 1.9412
Epoch 7/25


2025-01-18 18:05:31.984053: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 78644992 bytes after encountering the first element of size 78644992 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2074 - loss: 1.9173

...

13/13 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.9605 - loss: 0.1438 - val_accuracy: 0.2772 - val_loss: 5.0109
Epoch 25/25


2025-01-18 18:10:55.975519: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 78644992 bytes after encountering the first element of size 78644992 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9816 - loss: 0.0936

2025-01-18 18:11:11.608697: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 78644992 bytes after encountering the first element of size 78644992 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


13/13 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.9808 - loss: 0.0956 - val_accuracy: 0.2574 - val_loss: 3.9043


In [10]:
# Save the trained model
model.save("cat_emotion_model.h5")

print("Model training complete. Saved as 'cat_emotion_model.h5'.")

Model training complete. Saved as 'cat_emotion_model.h5'.
